# Own Custom CNN Architecture

### accuracy: 0.9590, val_accuracy: 0.9826

## Using  CascadeClassifier

In [2]:
# Import necessary libraries

import cv2
import keras
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [3]:
print(cv2.__version__)
print(keras.__version__)
print(np.__version__)

4.5.5
2.8.0
1.22.3


In [13]:
# Load the model
# model = load_model(filepath='model.h5')

model = load_model(filepath='resnet152v2.h5')


# This model accept below height, and width of the image because we train by this size

# image_width, image_height = (200, 200)

image_width, image_height = (224, 224) # For resnet152v2 model


# Load the CascadeClassifier

face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


# Capture the webcam

capture_video = cv2.VideoCapture(0)

img_count_full = 0

class_label = ""

while capture_video.isOpened():
    img_count_full += 1
    
    success, frame = capture_video.read()
    
    if success == True:
        # Convert to grayscale
        gray_img = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
        
        # Detect the faces
        faces = face_classifier.detectMultiScale(image=gray_img, scaleFactor=1.3, minNeighbors=3)
        
        img_count = 0
        
        for (x, y, width, height) in faces:
#             org = (x-10, y-10)
            img_count += 1
            
            color_face = frame[y : y + height, x : x + width] # Extract only face part
            
            cv2.imwrite(filename='faces/input/%d%dface.jpg' %(img_count_full, img_count), img=color_face)
            
            img = load_img(path='faces/input/%d%dface.jpg' %(img_count_full, img_count), target_size=(image_width, image_height))
            
            img2 = img_to_array(img=img) / 255
            
            img3 = np.expand_dims(img2, axis=0)
            
            predict_probability = model.predict(x=img3)
            print("Predict probability: ", predict_probability)
            
            predicted = np.argmax(predict_probability)
            print("Predicted: ", predicted)
            
            
            if predicted == 0:
#                 print("User with Mask predict", predict_probability[0][0])
                class_label = 'Mask'
#                 cv2.imwrite(filename="faces/with_mask/%d%dface.jpg" %(img_count_full, img_count), img=color_face)
            else:
#                 print("User without Mask predict", predict_probability[0][1])
                class_label = 'No Mask'
#                 cv2.imwrite(filename="faces/without_mask/%d%dface.jpg" %(img_count_full, img_count), img=color_face)
            
            cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+width, y+height), color=(0,255,255), thickness=2)
            cv2.rectangle(img=frame, pt1=(x-1, y-45), pt2=(x+width+1, y), color=(0,255,255), thickness=cv2.FILLED)
            cv2.putText(img=frame, text=class_label, org=(x, y-10),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=1, color=(255,0,255),
                        thickness=2, lineType=cv2.LINE_AA)
            
        # Display the image
        cv2.imshow(winname="Live face mask detection", mat=frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    else:
        break

capture_video.release()
cv2.destroyAllWindows()

Predict probability:  [[1.000000e+00 3.000466e-20]]
Predicted:  0
Predict probability:  [[1.000000e+00 7.058667e-26]]
Predicted:  0
Predict probability:  [[1.0000000e+00 2.2609554e-24]]
Predicted:  0
Predict probability:  [[1.000000e+00 7.056588e-20]]
Predicted:  0
Predict probability:  [[1.0000000e+00 1.1662015e-14]]
Predicted:  0
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[0. 1.]]
Predicted:  1
Predict probability:  [[1. 0.]]
Predicted:  0
Predict probability:  

## Using CVlib libraries

In [10]:
# Import necessary libraries

import cv2
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import cvlib


model = load_model(filepath='model.h5')


image_width, image_height = (200, 200)

# open webcam
webcam = cv2.VideoCapture(0)


img_count_full = 0

class_label = ""

if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    

# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

#     if not status:
#         print("Could not read frame")
#         exit()

    # apply face detection
    face, confidence = cvlib.detect_face(image=frame)

#     print(face)
#     print(confidence)

    img_count = 0
    
    # loop through detected faces
    for idx, f in enumerate(face):
        
        img_count += 1
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        color_face = frame[startY : startY + endY, startX : startX + endX] # Extract only face part
        
        cv2.imwrite(filename='faces/input/%d%dface.jpg' %(img_count_full, img_count), img=color_face)
            
        img = load_img(path='faces/input/%d%dface.jpg' %(img_count_full, img_count), target_size=(image_width, image_height))

        img = img_to_array(img=img) / 255

        img = np.expand_dims(img, axis=0)

        predict_probability = model.predict(x=img)
        print("Predicted Probability: ", predict_probability)
        
        predicted = np.argmax(predict_probability)
        print("Predicted: ", predicted)
            
        if predicted == 0:
#             print("User with Mask predict", predict_probability[0][0])
            class_label = 'Mask'
            cv2.imwrite(filename="faces/with_mask/%d%dface.jpg" %(img_count_full, img_count), img=color_face)
        else:
#             print("User without Mask predict", predict_probability[0][1])
            class_label = 'No Mask'
            cv2.imwrite(filename="faces/without_mask/%d%dface.jpg" %(img_count_full, img_count), img=color_face)

        cv2.rectangle(img=frame, pt1=(startX, startY), pt2=(endX, endY), color=(0,255,255), thickness=2)
        cv2.rectangle(img=frame, pt1=(startX, startY), pt2=(endX, endY), color=(0,255,255), thickness=cv2.FILLED)
        
        cv2.putText(img=frame, text=class_label, org=(startX, startY),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(255,0,255),
                    thickness=2, lineType=cv2.LINE_AA)

    # display output
    cv2.imshow(winname="Real-time face detection", mat=frame)
    
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows() 

Predicted Probability:  [[0.90306383 0.09693611]]
Predicted:  0
Predicted Probability:  [[0.8685867  0.13141333]]
Predicted:  0
Predicted Probability:  [[0.80059934 0.19940062]]
Predicted:  0
Predicted Probability:  [[0.8024585  0.19754143]]
Predicted:  0
Predicted Probability:  [[0.8682231  0.13177697]]
Predicted:  0
Predicted Probability:  [[0.8634282  0.13657185]]
Predicted:  0
Predicted Probability:  [[0.83249825 0.16750176]]
Predicted:  0
Predicted Probability:  [[0.8201821  0.17981796]]
Predicted:  0
Predicted Probability:  [[0.8738908  0.12610923]]
Predicted:  0
Predicted Probability:  [[0.87995803 0.12004201]]
Predicted:  0
Predicted Probability:  [[0.88743323 0.11256675]]
Predicted:  0
Predicted Probability:  [[0.86385727 0.13614275]]
Predicted:  0
Predicted Probability:  [[0.7932592  0.20674083]]
Predicted:  0
Predicted Probability:  [[0.7493843 0.2506157]]
Predicted:  0
Predicted Probability:  [[0.960091   0.03990896]]
Predicted:  0
Predicted Probability:  [[0.94899136 0.051